### Arguments

In [1]:
from dataclasses import dataclass

@dataclass
class args:
    coco_path: str="balloon"
    masks: bool=True
    batch_size: int=4
    num_workers: int=0
    hidden_dim: int=256
    backbone: str="resnet50"
    dilation: bool=True

args = args()   

### Get sample data

In [2]:
import torch
from torch.utils.data import (
    DataLoader, 
    RandomSampler, 
    SequentialSampler, 
    BatchSampler
)
import src.util.misc as utils
from src.datasets.coco import build as build_dataset

dataset_train = build_dataset(image_set='train', args=args)
dataset_val = build_dataset(image_set='val', args=args)

sampler_train = RandomSampler(dataset_train)
sampler_val = SequentialSampler(dataset_val)

batch_sampler_train = BatchSampler(
    sampler_train, 
    args.batch_size, 
    drop_last=True
)

data_loader_train = DataLoader(
    dataset_train, 
    batch_sampler=batch_sampler_train,
    collate_fn=utils.collate_fn, 
    num_workers=args.num_workers
)

data_loader_val = DataLoader(
    dataset_val, 
    args.batch_size, 
    sampler=sampler_val,
    drop_last=False, 
    collate_fn=utils.collate_fn, 
    num_workers=args.num_workers
)

for samples, targets in data_loader_train:
    if isinstance(samples, (list, torch.Tensor)):
        samples = utils.nested_tensor_from_tensor_list(samples)
    break

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


### Position Embedding

In [3]:
import math
import torch.nn as nn
from src.util.misc import NestedTensor

class PositionEmbeddingSine(nn.Module):
    """
    This is a more standard version of the position embedding, very similar to the one
    used by the Attention is all you need paper, generalized to work on images.
    """
    def __init__(self, num_pos_feats=64, temperature=10000, normalize=False, scale=None):
        super().__init__()
        self.num_pos_feats = num_pos_feats
        self.temperature = temperature
        self.normalize = normalize
        if scale is not None and normalize is False:
            raise ValueError("normalize should be True if scale is passed")
        if scale is None:
            scale = 2 * math.pi
        self.scale = scale

    def forward(self, tensor_list: NestedTensor):
        x = tensor_list.tensors
        mask = tensor_list.mask
        assert mask is not None
        not_mask = ~mask
        y_embed = not_mask.cumsum(1, dtype=torch.float32)
        x_embed = not_mask.cumsum(2, dtype=torch.float32)
        if self.normalize:
            eps = 1e-6
            y_embed = y_embed / (y_embed[:, -1:, :] + eps) * self.scale
            x_embed = x_embed / (x_embed[:, :, -1:] + eps) * self.scale

        dim_t = torch.arange(self.num_pos_feats, dtype=torch.float32, device=x.device)
        dim_t = self.temperature ** (2 * torch.floor(dim_t / 2) / self.num_pos_feats)

        pos_x = x_embed[:, :, :, None] / dim_t
        pos_y = y_embed[:, :, :, None] / dim_t
        pos_x = torch.stack((pos_x[:, :, :, 0::2].sin(), pos_x[:, :, :, 1::2].cos()), dim=4).flatten(3)
        pos_y = torch.stack((pos_y[:, :, :, 0::2].sin(), pos_y[:, :, :, 1::2].cos()), dim=4).flatten(3)
        pos = torch.cat((pos_y, pos_x), dim=3).permute(0, 3, 1, 2)
        return pos

In [4]:
import matplotlib.pyplot as plt
from ipywidgets import interact

N_steps = args.hidden_dim // 2
position_embedding = PositionEmbeddingSine(N_steps, normalize=True)
out = position_embedding(samples)
batch, _, n_x, n_y = out.shape

print(f"POS Shape: {out.shape}")

# graph
@interact
def show_position_embedding(batch=range(batch), x=(0, n_x-1), y=(0, n_y-1)):
    plt.plot(out[batch, :, x, y])
    plt.show()

POS Shape: torch.Size([4, 256, 820, 966])


interactive(children=(Dropdown(description='batch', options=(0, 1, 2, 3), value=0), IntSlider(value=409, descr…

### Backbone

In [5]:
import torchvision
import torch.nn.functional as F
from torchvision.models._utils import IntermediateLayerGetter
from src.util.misc import NestedTensor, is_main_process

class FrozenBatchNorm2d(torch.nn.Module):
    """
    BatchNorm2d where the batch statistics and the affine parameters are fixed.
    Copy-paste from torchvision.misc.ops with added eps before rqsrt,
    without which any other models than torchvision.models.resnet[18,34,50,101]
    produce nans.
    """

    def __init__(self, n):
        super(FrozenBatchNorm2d, self).__init__()
        self.register_buffer("weight", torch.ones(n))
        self.register_buffer("bias", torch.zeros(n))
        self.register_buffer("running_mean", torch.zeros(n))
        self.register_buffer("running_var", torch.ones(n))

    def _load_from_state_dict(self, state_dict, prefix, local_metadata, strict,
                              missing_keys, unexpected_keys, error_msgs):
        num_batches_tracked_key = prefix + 'num_batches_tracked'
        if num_batches_tracked_key in state_dict:
            del state_dict[num_batches_tracked_key]

        super(FrozenBatchNorm2d, self)._load_from_state_dict(
            state_dict, prefix, local_metadata, strict,
            missing_keys, unexpected_keys, error_msgs)

    def forward(self, x):
        # move reshapes to the beginning
        # to make it fuser-friendly
        w = self.weight.reshape(1, -1, 1, 1)
        b = self.bias.reshape(1, -1, 1, 1)
        rv = self.running_var.reshape(1, -1, 1, 1)
        rm = self.running_mean.reshape(1, -1, 1, 1)
        eps = 1e-5
        scale = w * (rv + eps).rsqrt()
        bias = b - rm * scale
        return x * scale + bias

class BackboneBase(nn.Module):
    def __init__(self, backbone: nn.Module, train_backbone: bool, num_channels: int, return_interm_layers: bool):
        super().__init__()
        for name, parameter in backbone.named_parameters():
            if not train_backbone or 'layer2' not in name and 'layer3' not in name and 'layer4' not in name:
                parameter.requires_grad_(False)
        if return_interm_layers:
            return_layers = {"layer1": "0", "layer2": "1", "layer3": "2", "layer4": "3"}
        else:
            return_layers = {'layer4': "0"}
        self.body = IntermediateLayerGetter(backbone, return_layers=return_layers)
        self.num_channels = num_channels

    def forward(self, tensor_list: NestedTensor):
        xs = self.body(tensor_list.tensors)
        out: Dict[str, NestedTensor] = {}
        for name, x in xs.items():
            m = tensor_list.mask
            assert m is not None
            mask = F.interpolate(m[None].float(), size=x.shape[-2:]).to(torch.bool)[0]
            out[name] = NestedTensor(x, mask)
        return out

class Backbone(BackboneBase):
    """ResNet backbone with frozen BatchNorm."""
    def __init__(self, name: str,
                 train_backbone: bool,
                 return_interm_layers: bool,
                 dilation: bool):
        backbone = getattr(torchvision.models, name)(
            replace_stride_with_dilation=[False, False, dilation],
            pretrained=is_main_process(), norm_layer=FrozenBatchNorm2d)
        num_channels = 512 if name in ('resnet18', 'resnet34') else 2048
        super().__init__(backbone, train_backbone, num_channels, return_interm_layers)

class Joiner(nn.Sequential):
    def __init__(self, backbone, position_embedding):
        super().__init__(backbone, position_embedding)

    def forward(self, tensor_list: NestedTensor):
        xs = self[0](tensor_list)
        out: List[NestedTensor] = []
        pos = []
        for name, x in xs.items():
            out.append(x)
            # position encoding
            pos.append(self[1](x).to(x.tensors.dtype))

        return out, pos

In [6]:
train_backbone = True
return_interm_layers = args.masks
backbone = Backbone(args.backbone, train_backbone, return_interm_layers, args.dilation)
model = Joiner(backbone, position_embedding)
model.num_channels = backbone.num_channels
out_list, pos_list = model(samples)

In [7]:
def inspect(out, pos):
    x, mask = out.decompose()
    x = x.detach()
    mask = mask.detach()
    print(f"{'Img. shape':12s}: {x.shape}")
    print(f"{'Mask shape':12s}: {mask.shape}")
    print(f"{'Pos. shape':12s}: {pos.shape}")
    
    batch, n_c, _, _ = x.shape
    _, n_p, _, _ = pos.shape
    @interact
    def show_result(batch=(0, batch-1), channel=(0, n_c-1), pos_dim=(0, n_p-1)):
        plt.figure(figsize=(20, 10))
        
        plt.subplot(1, 3, 1)
        plt.title("Image")
        plt.imshow(x[batch, channel, :, :], cmap="coolwarm")

        plt.subplot(1, 3, 2)
        plt.title("Mask")
        plt.imshow(mask[batch, :, :], cmap="coolwarm")

        plt.subplot(1, 3, 3)
        plt.title("Position")
        plt.imshow(pos[batch, pos_dim, :, :], cmap="coolwarm")

        plt.show()

In [8]:
for out, pos in zip(out_list, pos_list):
    inspect(out, pos)

Img. shape  : torch.Size([4, 256, 205, 242])
Mask shape  : torch.Size([4, 205, 242])
Pos. shape  : torch.Size([4, 256, 205, 242])


interactive(children=(IntSlider(value=1, description='batch', max=3), IntSlider(value=127, description='channe…

Img. shape  : torch.Size([4, 512, 103, 121])
Mask shape  : torch.Size([4, 103, 121])
Pos. shape  : torch.Size([4, 256, 103, 121])


interactive(children=(IntSlider(value=1, description='batch', max=3), IntSlider(value=255, description='channe…

Img. shape  : torch.Size([4, 1024, 52, 61])
Mask shape  : torch.Size([4, 52, 61])
Pos. shape  : torch.Size([4, 256, 52, 61])


interactive(children=(IntSlider(value=1, description='batch', max=3), IntSlider(value=511, description='channe…

Img. shape  : torch.Size([4, 2048, 52, 61])
Mask shape  : torch.Size([4, 52, 61])
Pos. shape  : torch.Size([4, 256, 52, 61])


interactive(children=(IntSlider(value=1, description='batch', max=3), IntSlider(value=1023, description='chann…